In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/Colab Notebooks/dezeenAIsearch

/content/gdrive/My Drive/Colab Notebooks/dezeenAIsearch


In [10]:
import requests
from bs4 import BeautifulSoup
import urllib.request
from tqdm import tqdm
import timeit
import numpy as np
import pandas as pd
import sys

In [4]:
start = timeit.default_timer() # start the times

ids = []
titles = []
urls = []

print('\nStarting. This might take a few minutes to complete...\n')

for n in range(1, 500):

  url = 'https://www.dezeen.com/interiors/page/'+str(n)
  response = requests.get(url)
  if response.status_code != 200:
    print('\nCrawled a total of {} pages. Finishing.'.format(n-1))
    break

  soup = BeautifulSoup(response.content, 'html.parser')
  articles = soup.find_all('article', attrs={'data-article-id': True})

  for article in articles:
    ids.append(article['data-article-id'])
    titles.append(article.h3.string)
    urls.append(article.a['href'])

stop = timeit.default_timer() # stop the timer
print('Runtime: {} seconds.'.format(stop-start))


Starting. This might take a few minutes to complete...


Crawled a total of 248 pages. Finishing.
Runtime: 541.9636806220005 seconds.


In [6]:
articles_df = pd.DataFrame(
    data = list(zip(ids, titles, urls)),
    columns = ['id', 'title', 'url'])

articles_df

,id,title,url
0,1588230,Daosheng Design creates monochromatic bar with...,https://www.dezeen.com/2020/11/19/the-flow-of-...
1,1588111,Issey Miyake store in Osaka is splashed with w...,https://www.dezeen.com/2020/11/19/issey-miyake...
2,1586860,Nook Pod is a gabled workspace,https://www.dezeen.com/2020/11/18/nook-pod-dez...
3,1587217,Project #13 is an office for Studio Wills + Ar...,https://www.dezeen.com/2020/11/17/project-13-h...
4,1586339,AHEAD Europe 2020 awards winners announced in ...,https://www.dezeen.com/2020/11/16/ahead-europe...
...,...,...,...
4946,70,Marcel Wanders launches Crochet Chair,https://www.dezeen.com/2006/12/10/marcel-wande...
4947,67,WOKmedia show at Design Miami,https://www.dezeen.com/2006/12/10/wokmedia-sho...
4948,54,Zaha Hadid furniture exhibited in New York,https://www.dezeen.com/2006/12/07/zaha-hadid-f...
4949,48,Thomas Heatherwick beach cafe takes shape,https://www.dezeen.com/2006/12/04/thomas-heath...


In [7]:
start = timeit.default_timer() # start the times

print('\nStarting. This might take around two hours to complete...\n')

article_images = []

for url in tqdm(articles_df['url']):
  
  response = requests.get(url)
  if response.status_code != 200:
      print('\nPage not available.'.format(n-1))
  soup = BeautifulSoup(response.content, 'html.parser')
  figures = soup.find_all('figure', attrs={'id': True})

  imgs = []
  for figure in figures:
    if figure.img is not None:
      imgs.append(figure.img['data-src'])
  
  article_images.append(imgs)
    
stop = timeit.default_timer() # stop the timer
print('Runtime: {} seconds.'.format(stop-start))

  0%|          | 0/4951 [00:00<?, ?it/s]


Starting. This might take around two hours to complete...



100%|██████████| 4951/4951 [1:52:00<00:00,  1.36s/it]

Runtime: 6720.473755998006 seconds.


In [8]:
articles_df['images'] = article_images
articles_df

,id,title,url,images
0,1588230,Daosheng Design creates monochromatic bar with...,https://www.dezeen.com/2020/11/19/the-flow-of-...,[https://static.dezeen.com/uploads/2020/11/the...
1,1588111,Issey Miyake store in Osaka is splashed with w...,https://www.dezeen.com/2020/11/19/issey-miyake...,[https://static.dezeen.com/uploads/2020/11/iss...
2,1586860,Nook Pod is a gabled workspace,https://www.dezeen.com/2020/11/18/nook-pod-dez...,[https://static.dezeen.com/uploads/2020/11/noo...
3,1587217,Project #13 is an office for Studio Wills + Ar...,https://www.dezeen.com/2020/11/17/project-13-h...,[https://static.dezeen.com/uploads/2020/11/pro...
4,1586339,AHEAD Europe 2020 awards winners announced in ...,https://www.dezeen.com/2020/11/16/ahead-europe...,[]
...,...,...,...,...
4946,70,Marcel Wanders launches Crochet Chair,https://www.dezeen.com/2006/12/10/marcel-wande...,[https://static.dezeen.com/uploads/2006/12/Mar...
4947,67,WOKmedia show at Design Miami,https://www.dezeen.com/2006/12/10/wokmedia-sho...,[https://static.dezeen.com/uploads/2006/12/WOK...
4948,54,Zaha Hadid furniture exhibited in New York,https://www.dezeen.com/2006/12/07/zaha-hadid-f...,[https://static.dezeen.com/uploads/2006/12/Zah...
4949,48,Thomas Heatherwick beach cafe takes shape,https://www.dezeen.com/2006/12/04/thomas-heath...,[https://static.dezeen.com/uploads/2006/12/Tho...


In [13]:
sys.setrecursionlimit(10000)

In [14]:
articles_df.to_pickle('files/articles.pkl')

In [17]:
articles_df.to_hdf('files/articles.h5', key= 'df')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['id', 'title', 'url', 'images'], dtype='object')]

  encoding=encoding,
